# Electronic Structure Calculations using Generalized Ising Hamiltonians

Your final task is to calculate the ground state energy of a hydrogen molecule using a Generalized Ising Hamiltonian:

$$ 
H = E_0 + \sum_i h_i\sigma_i + \sum_{ij} J_{ij}\sigma_i\sigma_j
+ \sum_{ijk} K_{ijk}\sigma_i\sigma_j\sigma_k
+ \sum_{ijkl} L_{ijkl}\sigma_i\sigma_j\sigma_k\sigma_l
+ \cdots
$$

where the Hamiltonian parameters ($E_0, h_i, J_{ij}, K_{ijk}, L_{ijkl}$) will be provided to you by the `read_generalized_ising_hamiltonian` function below.

These Ising Hamiltonians were produced using the *Iterative Qubit Coupled Cluster* method (https://arxiv.org/abs/1906.11192).

The $H_2$ Ising Hamiltonian only needs 4 spins to fully describe the ground state, hence our Hamiltonian only needs a 4-point interaction at most.

In [1]:
import numpy as np

from abstract_ising import AbstractIsing
from ising_animator import IsingAnimator

%matplotlib inline

In [2]:
def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]
    
    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms

In [7]:
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65.inp")

#print(E0)
#print(h)
#print(J)
#print(K)
#print(L)

0.03775117547636375
[0.         0.18262644 0.12584137 0.19100352]
[[ 0.          0.          0.12584137  0.00837708]
 [ 0.          0.          0.          0.1729761 ]
 [ 0.          0.          0.         -0.27288405]
 [ 0.          0.          0.          0.        ]]
[[[ 0.          0.          0.          0.        ]
  [ 0.          0.         -0.26450696  0.17866778]
  [ 0.          0.          0.         -0.00837708]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.16992098]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.     

First, you must write a class that describes the Generalized Ising Model given the arrays containing the Hamiltonian parameters. You will need to write a function which computes energy of the stored spin configuration (see the previous tasks for inspiration), as well as a function which computes the change in energy resulting from a single-spin-flip (you could of course use the naive approach and simply compute the energy of two spin configurations and subtract one from the other).

In [4]:
# Write a class to describe the generalized Ising Model given its Hamiltonian parameters
# Only need 4 spins
class GeneralizedIsingModel(AbstractIsing):
    def __init__(self, N, E0, h, J, K, L):
        self.N = N
        self.E0 = E0
        self.h = h
        self.J = J
        self.K = K
        self.L = L
        self.num_spins = self.N
        
        # initialize system at infinite temperature
        # i.e. spins are completely random and uncorrelated
        self.spins = 2*(np.random.rand(self.N) < 0.5) - 1
    
    def energy(self):
        # hint: np.einsum may be helpful here
        """Returns the energy of the current spin configuration"""
        spins = self.spins if spins is None else spins
        #interactions = self.bonds * spins * np.roll(spins, 1, axis=-1)
        #total = self.J * np.sum(interactions, axis=-1)
        # E0 + h_i + J_ij + K_ijk + L_ijkl
        total = self.E0 + np.einsum('i',h) + np.einsum('ij->i',J) + np.einsum('ijk->i', K) + np.einsum('ijkl->i', L)
        return total
        pass
    
    def energy_diff(self, *coords):
        """Returns the energy difference resulting from flipping the i'th site"""
        # sum the nearest neighbour sites
        
        nn_sum = (
            self.bonds[i] * self.spins[i-1] 
            + self.bonds[(i+1)%self.N] * self.spins[(i+1) % self.N] 
        )

        return -2 * self.J * self.spins[i] * nn_sum
    
    def rand_site(self):
        """Selects a site in the lattice at random"""
        return (np.random.randint(self.N),)

Next you'll run a Monte Carlo simulation for this model at some finite temperature for 1000 steps, printing out the energy of the state every so often

Now, apply (one of) the annealing procedure(s) you came up with in the previous task to this problem to find a ground state of the system:

In [6]:
# your annealing code here
def annealing_schedule(steps, start_temp, final_temp):
    N = steps
    T_i = start_temp
    T_f = final_temp
    for t in range(N):
        # Annealing
        T = T_i * ((T_f/T_i) ** (t/N))
        E = ising.mc_step(T=T)

        if t % 50 == 0:
            print(E)
            
    return E

# Steps in MC
N = 1000

# Starting temperature
T_i = 1000

# Final Temperature
T_f = 0.01

print("\n=== Annealing Energy Calculation for 4 spins ===")
annealed_energy = annealing_schedule(N,T_i,T_f)

Finally, iterate over the entire spin configuration space (this is tractable since we only have 4 spins) to find the exact ground state energy. Compare this energy to the one you found above using your annealer.

In [7]:
# exact ground state calculation

Now, clean up your code a little, and write a for-loop that iterates over all the different values of the Hydrogen seperation distance $R$ available in the `hamiltonians` directory. 

For each $R$ you must:
- Read in the associated Ising Hamiltonian
- Perform an annealed Monte Carlo simulation to find a candidate ground state energy
- Compute the exact ground state energy
- Compare the two results